In [ ]:
# 가장 먼저 DQN에 대한 개념을 알아두어야 한다.
# 골빈해커책 224~228의 내용을 잘 확인하고 특히 228 페이지의 그림을 잘 기억해두자.
# 현 소스코드인 에이전트는 두가지 역활을 한다.
# 1) 게임으로부터 게임의 상태를 기본 신경망으로 전달한다.
# 2) 신경망에서 판단한 행동을 게임에 적용한다.
# 즉, 이 코드에서 신경망은 구현하지 않는다.

In [ ]:
import tensorflow as tf
import numpy as np
import random
import time

from game import Game
from model import DQN
# 파이썬 문법
# 위의 코드는 아래 두 행동을 하고 있다.
# 1) game.py 파일로부터 Game 클래스를 임포트한다.
# 2) model.py 파일로부터 DQN 클래스를 임포트한다.

In [ ]:
# 에이전트의 실행 모드는 1)학습모드 와 2)게임실행모드(replay) 로 나뉜다.
# 1) 학습모드 : 게임을 화면에 보여주지 않고 빠르게 실행하여 학습속도를 높인다.
# 2) 게임실행모드 : 학습한 결과를 이용하여 게임을 진행하면서 화면에도 출력한다.
# 이것을 위해 tf.app.flags를 이용한다.
tf.app.flags.DEFINE_boolean(
    "train",
    False,
    "학습모드. 게임을 화면에 보여주지 않습니다.")
FLAGS = tf.app.flags.FLAGS

In [ ]:
# 이번에는 하이퍼 파라미터들을 설정한다.
HP_MAX_EPISODE = 10000           # 최대로 학습할 게임 횟수
HP_TARGET_UPDATE_INTERVAL = 1000 # 학습을 n만큼 진행할때마다 한번씩 
HP_TRAIN_INTERVAL = 4            # n번의 게임 프레임 마다 한번씩 학습하라.
HP_OBSERVE = 100                 # 게임 프레임이 n 값까지 도달한 이후에 학습을 시작한다.

# HP_OBSERVE가 존재하는 이후는 게임 시작 시점의 학습 데이터가 적을때는
# 학습을 진행해도 효과가 크기 않기 때문이다.

In [ ]:
# 이제는 게임에 필요한 설정을 세팅해보도록 하자.

# 행동값 -> 
# 0 : 좌
# 1 : 유지
# 2 : 우
NUM_ACTION = 3
SCREEN_WIDTH = 6
SCREEN_HEIGHT = 10

In [ ]:
def train():
    
    # 텐서플로 세션과 게임 객체, DQN 모델 객체를 생성한다.
    print("뇌세포 깨우는 중...")
    sess = tf.Session()
    
    # 게임 내에는 화면 크기와 게임을 보여줄 것인지의 여부(False)를 인자로 준다.
    game = Game(SCREEN_WIDTH, SCREEN_HEIGHT, show_game=False)
    # DQN 모델 객체에는 텐서플로 세션과 화면크기, 선택할 행동의 갯수를 넣어준다.
    brain = DQN(sess, SCREEN_WIDTH, SCREEN_HEIGHT, NUM_ACTION)
    
    # 이 값은 게임을 한판하고 얻는 점수를 저장하고, 확인하기 위한 텐서이다.
    # 차후에는 10판에 한번씩 로그를 저장하고, 그때 rewards의 평균을 저장할 것이다.
    rewards = tf.placeholder(tf.float32, [None])
    tf.summary.scalar('avg.reward/ep.', tf.reduce_mean(rewards))
    
    saver = tf.train.Saver()
    sess.run(tf.global_variables_initializer())
    
    writer = tf.summary.FileWriter('logs', sess.graph)
    summary_merged = tf.summary.merge_all()
    
    